Complete AI Agent orchestration with langgraph

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

Triage chain

In [8]:
from pydantic import BaseModel, Field
from typing import Literal

class TriageOut(BaseModel):
    decision: Literal["SELF_RESOLVE","ASK_FOR_INFO","OPEN_TICKET"]
    urgency: Literal["LOW", "MEDIUM", "HIGH"]
    missing_fields: list[str] = Field(default_factory=list)

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [10]:
triage_Prompt = ("""
                    Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento.
Dada a mensagem do usuário, retorne SOMENTE um JSON com:
{
  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",
  "urgencia": "BAIXA" | "MEDIA" | "ALTA",
  "campos_faltantes": ["..."]
}

Regras de Decisão:
- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").
- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral"). 
  # Melhoria: Instrução para preencher campos_faltantes de forma útil.
  Ao usar esta decisão, preencha "campos_faltantes" com os tópicos que precisam de esclarecimento (Ex: "nome da política", "descrição do problema").
- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").

Regras de Urgência:
- **ALTA**: O usuário relata um problema que o impede completamente de trabalhar, menciona risco de segurança, legal ou perda de dados. (Ex: "Não consigo acessar o sistema para trabalhar", "Recebi um e-mail de phishing e cliquei no link").
- **MEDIA**: O usuário tem um problema que dificulta o trabalho mas não o impede, ou tem um prazo se aproximando. (Ex: "Preciso da aprovação para uma viagem na próxima semana").
- **BAIXA**: Perguntas gerais, dúvidas sobre políticas ou solicitações que não têm um prazo imediato. (Ex: "Como funciona a política de férias?", "Tenho uma dúvida sobre reembolso").

Analise a mensagem e decida a ação e urgência mais apropriadas com base nas regras acima.
""")

In [11]:
from langchain_core.messages import SystemMessage, HumanMessage

triage_chain = llm.with_structured_output(TriageOut)

def triage(message:str) ->dict:
    output: TriageOut = triage_chain.invoke([
        SystemMessage(content=triage_Prompt),
        HumanMessage(content=message)
    ])
    return output.model_dump()

RAG chain

In [12]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []
data_path = Path('text_docs/')
for d in data_path.glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(d))
        docs.extend(loader.load())
        print(f"file loaded succesifuly: {d.name}")
    except Exception as e:
        print(f"failed to load the file {d.name}: {e}")


file loaded succesifuly: Política de Reembolsos (Viagens e Despesas).pdf
file loaded succesifuly: Política de Uso de E-mail e Segurança da Informação.pdf
file loaded succesifuly: Políticas de Home Office.pdf


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [15]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_type= 'similarity_score_threshold',
                                     search_kwargs = {'score_threshold':0.3,'k':4})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert HR and IT assistant for the company 'Torres Devs'. "
               "Your main task is to answer employee questions based ONLY on the context provided. "
               "Be polite and professional in your responses. "
               "If the information to answer the question is not in the provided context, "
               "clearly state that you cannot find the answer in the company's documents."),
    
    ("user", "Based on our company's policies, please answer the following question:\n\nQuestion: {input}\n\nContext:\n{context}")
])

document_chain = create_stuff_documents_chain(llm, rag_prompt)

In [ ]:
def ask_question_rag(question:str) -> dict:
    related_docs = retriever.invoke(question)

    if not related_docs:
        return {"answer": "cannot find the answer in the company's documents",
                "citations":[],
                "context_found": False}
    
    answer = document_chain.invoke({"input": question,
                                    "context": related_docs})
    text = (answer or "").strip()

    if text.rstrip(".!?") == "cannot find the answer in the company's documents":
        return {"answer": "cannot find the answer in the company's documents",
                "citations":[],
                "context_found": False}
    return {"answer": text,
                "citations":related_docs,
                "context_found": True}

CODIGO DA AULA

In [ ]:
from typing import TypedDict, Optional, Dict, List

class AgentState(TypedDict, total = False):
    mensagem: str
    triage: dict
    answer: Optional[str]
    citations: List[dict]
    rag_succes: bool
    final_action: str

In [ ]:
def triage_node(state: AgentState) -> AgentState:
    print("Executing the triaging node")
    return {"triage": triage(state["message"])}


In [ ]:
def self_resolve_node(state: AgentState) -> AgentState:
    print("Executing the self resolve node")
    rag_answer